In [ ]:
from openslide import OpenSlide
import PIL

In [ ]:
wsi = OpenSlide('/path to wsi/')

In [ ]:
import geojson

In [ ]:
with open('/path to wsi annotation geojson/', 'r') as f:
    manual_annotations = geojson.load(f)

In [ ]:
import geopandas as gpd
manual_annotations_df = gpd.GeoDataFrame.from_features(manual_annotations).scale(yfact=-1, origin=(0,0)).scale(xfact=1/wsi.level_downsamples[-2], yfact=1/wsi.level_downsamples[-2], origin=(0, 0))

In [ ]:
wsi.level_downsamples

In [ ]:
wsi.level_dimensions

In [ ]:
manual_annotations_df.plot()

In [ ]:
manual_annotations_df

In [ ]:
import numpy as np
import logging
from matplotlib import pyplot as plt

def plot_slide_and_polygons(
    anomalies: gpd.GeoSeries,
    thumbnail: np.array,
    figsize_factor: int = 20,
) -> None:
    """
    Plot a collection of polygons on top of a thumbnail of a Whole Slide Image.

    Args:
        anomalies (geopandas.GeoSeries): A collection of polygons (for example,
        representing anomalous regions on the slide image).

        thumbnail (np.array): A thumbnail on top of which to plot anomalies.

        figsize_factor (int, optional): Factor to magnify the entire plot
        by at the end of the plotting process (e.g., to make details easier
        to see in certain text editors). Defaults to 1.

        show_all (bool, optional): Whether to plot all polygons from anomalies.
        If false, only polygons whose indexes are given in
        anomalies_to_highlight are plotted. Defaults to True.

        fill_in (bool, optional): Whether to plot polygons as filled-in shapes
        (True) or as outlines (False). Defaults to True.

        show_axes (bool, optional): Whether to display matplotlib axes.
        Defaults to True.

    Returns:
        None: (Nothing is returned, but a plot is created and displayed)
    """
    thumbnail_dimensions = thumbnail.size
    
    logging.info("Plotting...")
    thumbnail_dimension_ratio = min(thumbnail_dimensions) / max(
        thumbnail_dimensions
    )
    figsize = tuple(
        [x * figsize_factor for x in (1, thumbnail_dimension_ratio)]
    )

    _, ax = plt.subplots(1, 1, figsize=(figsize[0], figsize[1]))

    anomalies.exterior.plot(ax=ax, cmap="gist_rainbow")

    ax.imshow(
        thumbnail,
        aspect="auto",
        interpolation="bilinear",
        extent=(
            0,  # Left
            thumbnail_dimensions[0],  # Right
            -thumbnail_dimensions[1],  # Bottom
            0,  # Top
        ),
        # origin="upper",
        alpha=0.75,
        cmap="gray",
    )


In [ ]:
wsi.level_dimensions

In [ ]:
thumbnail = wsi.get_thumbnail(wsi.level_dimensions[-2])

In [ ]:
thumbnail

In [ ]:
manual_annotations_df['geometry']

In [ ]:
plot_slide_and_polygons(
    anomalies=manual_annotations_df,
    thumbnail=thumbnail
)